# Transformations of 2D image

Assuming the image is represent by 2D array:

Coordinate matrix:

```txt
(0, 0) (1, 0) (2, 0) (3, 0)
(0, 1) (1, 1) (2, 1) (3, 1)
(0, 2) (1, 2) (2, 2) (3, 2)
(0, 3) (1, 3) (2, 3) (3, 3)
(0, 4) (1, 4) (2, 4) (3, 4)
```

## Representation

Possible representation that kind of table:
* as 1D table
* as table of tables
* as **dictionary** with tuple as keys
* as **set** of tuples (representing coordinates) if picture is "bitmap" (contain only two colours)

In Python the two last options are worth considering.

In [1]:
def load_pixels(image: list) -> tuple:
    for row, line in enumerate(image):
        yield from ((column, row, c) for column, c in enumerate(line))

def to_set_image(image: list) -> set:
    return set((column, row) for column, row, c in load_pixels(image) if c == '#')

def to_dict_image(image: list) -> dict:
    return {(column, row): c for column, row, c in load_pixels(image)}

def print_set_image(image: set):
    max_column = max(column for column, _ in image)
    max_row = max(row for _, row in image)

    for row in range(0, max_row + 1):
        for column in range(0, max_column + 1):
            print('#' if (column, row) in image else ' ', end='')
        print()

def print_dict_image(image: dict):
    max_column = max(column for column, _ in image.keys())
    max_row = max(row for _, row in image.keys())

    for row in range(0, max_row + 1):
        for column in range(0, max_column + 1):
            print(image[column, row], end='')
        print()

## Turn 90 degrees

### Right

We want to achieve the following transformation of coordinates:

```txt
(0, 4) (0, 3) (0, 2) (0, 1) (0, 0)
(1, 4) (1, 3) (1, 2) (1, 1) (1, 0)
(2, 4) (2, 3) (2, 2) (2, 1) (2, 0)
(3, 4) (3, 3) (3, 2) (3, 1) (3, 0)
```

So as we can see, it comes to:

$$ (x_r, y_r) => (size_y - y - 1, x) $$

### Left

We want to achieve the following transformation of coordinates:

```txt
(3, 0) (3, 1) (3, 2) (3, 3) (3, 4)
(2, 0) (2, 1) (2, 2) (2, 3) (2, 4)
(1, 0) (1, 1) (1, 2) (1, 3) (1, 4)
(0, 0) (0, 1) (0, 2) (0, 3) (0, 4)
```

It comes to:

$$ (x_l, y_l) => (y, size_x - x - 1) $$

In [2]:
def right_90_transform(x: int, y: int, size_x: int, size_y: int) -> tuple:
    return (size_y - y - 1, x)

def left_90_transform(x: int, y: int, size_x: int, size_y: int) -> tuple:
    return (y, size_x - x - 1)

## Flips

### Vertical

```txt
(0, 4) (1, 4) (2, 4) (3, 4)
(0, 3) (1, 3) (2, 3) (3, 3)
(0, 2) (1, 2) (2, 2) (3, 2)
(0, 1) (1, 1) (2, 1) (3, 1)
(0, 0) (1, 0) (2, 0) (3, 0)
```

It comes to:

$$ (x_h, y_h) => (x, size_y - y - 1) $$

### Horizontal

```txt
(3, 0) (2, 0) (1, 0) (0, 0)
(3, 1) (2, 1) (1, 1) (0, 1)
(3, 2) (2, 2) (1, 2) (0, 2)
(3, 3) (2, 3) (1, 3) (0, 3)
(3, 4) (2, 4) (1, 4) (0, 4)
```

It comes to:

$$ (x_v, y_v) => (size_x - x - 1, y) $$

In [3]:
def flip_vertical(x: int, y: int, size_x: int, size_y: int) -> list:
    return (x, size_y - y - 1)

def flip_horizontal(x: int, y: int, size_x: int, size_y: int) -> list:
    return (size_x - x - 1, y)

In [4]:
arrow = to_set_image('''# # 
  ##
####'''.splitlines())

def transform_set_image(transform, picture: set, size_x: int, size_y: int) -> set:
    return set(transform(x, y, size_x, size_y) for x, y in picture)

print('Original picture:\n')
print_set_image(arrow)
print()

print('Rotate 90 right of original:\n')
print_set_image(transform_set_image(right_90_transform, arrow, 4, 3))
print()

print('Rotate 90 left of original:\n')
print_set_image(transform_set_image(left_90_transform, arrow, 4, 3))
print()

print('Flip vertical of original:\n')
print_set_image(transform_set_image(flip_vertical, arrow, 4, 3))
print()

print('Flip horizontal of original:\n')
print_set_image(transform_set_image(flip_horizontal, arrow, 4, 3))
print()


Original picture:

# # 
  ##
####

Rotate 90 right of original:

# #
#  
###
## 

Rotate 90 left of original:

 ##
###
  #
# #

Flip vertical of original:

####
  ##
# # 

Flip horizontal of original:

 # #
##  
####



In [5]:
alphabet = to_dict_image('''abcd
efgh
hijk'''.splitlines())

def transform_dict_image(transform, picture: set, size_x: int, size_y: int)  -> set:
    return {transform(x, y, size_x, size_y):picture[x, y] for x, y in picture.keys()}

print('Original picture:\n')
print_dict_image(alphabet)
print()

print('Rotate 90 right:\n')
print_dict_image(transform_dict_image(right_90_transform, alphabet, 4, 3))
print()

print('Rotate 90 left:\n')
print_dict_image(transform_dict_image(left_90_transform, alphabet, 4, 3))
print()

print('Flip vertical of original:\n')
print_dict_image(transform_dict_image(flip_vertical, alphabet, 4, 3))
print()

print('Flip horizontal of original:\n')
print_dict_image(transform_dict_image(flip_horizontal, alphabet, 4, 3))
print()


Original picture:

abcd
efgh
hijk

Rotate 90 right:

hea
ifb
jgc
khd

Rotate 90 left:

dhk
cgj
bfi
aeh

Flip vertical of original:

hijk
efgh
abcd

Flip horizontal of original:

dcba
hgfe
kjih

